In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from func_test_models import test_darts_impl, test_tensorflow_impl

np.set_printoptions(precision = 4)

In [ ]:
pathes_household = {"gas": "./data/Generated Data/master_competitor_market_prices_gas.csv", 
                    "power": "./data/Generated Data/master_competitor_market_prices_power.csv"}
pathes_market = {"gas": "./data/Generated Data/gas_prices.csv", 
                 "power": "./data/Generated Data/power_prices.csv"}

In [ ]:
energy = "power"

path_household = pathes_household[energy]
path_market = pathes_market[energy]

dtypes = {"date_valid_from": str, "date_valid_to": str, "consumption_range_kwh": np.int32, "price_year_eur": np.float64, "post_code": str, "rank": np.float16}
usecols = [name for name, _ in dtypes.items()]

household = pd.read_csv(path_household, usecols=usecols, dtype = dtypes)
market_data = pd.read_csv(path_market, header = [0], parse_dates=["Dates"])

market_data.set_index("Dates", inplace = True)
market_data["moving_average"] = market_data.avg_price.ewm(alpha=0.1, adjust=False).mean()
household["price_kwh"] = household["price_year_eur"]/household["consumption_range_kwh"]
household["date_valid_from"] = pd.to_datetime(household["date_valid_from"])
household["date_valid_to"] = pd.to_datetime(household["date_valid_to"])

postCodes = household.post_code.unique()
consRanges = household.consumption_range_kwh.unique()

In [ ]:
shift = 5
input_width = 10
rank_range = (6,10)
consRanges_test_models = consRanges[(consRanges<=5000) & (consRanges>=3000)]
postCodes_test_models = postCodes[43:47]

print(consRanges_test_models, postCodes_test_models)
print(f"{np.count_nonzero(consRanges_test_models)*np.count_nonzero(postCodes_test_models)} calculations to be done")

In [ ]:

mse_darts_df, mape_darts_df = test_darts_impl(shift = shift, 
                                consRanges=consRanges_test_models, 
                                postCodes=postCodes_test_models,
                                household_data=household,
                                market_data=market_data,
                                rank_range=rank_range)


In [ ]:

mse_tensorflow_df, mape_tensorflow_df = test_tensorflow_impl(shift = shift, 
                                input_width=input_width, 
                                consRanges=consRanges_test_models, 
                                postCodes=postCodes_test_models,
                                rank_range=rank_range,
                                household_data=household,
                                market_data=market_data)


In [ ]:
mape_darts_df.describe()

,baseline,LSTM,regression
count,28.0,28.000000,28.000000
mean,0.0,1.736733,1.570565
std,0.0,0.390016,0.586608
min,0.0,1.033151,0.672131
25%,0.0,1.416092,1.115775
50%,0.0,1.859008,1.473004
75%,0.0,2.020265,2.091430
max,0.0,2.324475,2.605242


In [ ]:
mse_darts_df.describe()

,baseline,LSTM,regression
count,28.0,28.000000,28.000000
mean,0.0,0.000226,0.000202
std,0.0,0.000090,0.000138
min,0.0,0.000079,0.000035
25%,0.0,0.000142,0.000092
50%,0.0,0.000238,0.000147
75%,0.0,0.000293,0.000321
max,0.0,0.000363,0.000501


In [ ]:
mape_tensorflow_df.describe()

,baseline,dense,convolutional,LSTM
count,28.000000,28.000000,28.000000,28.000000
mean,4.651255,17.108397,10.937198,3.367585
std,0.336317,10.427432,6.459199,1.885085
min,4.122550,10.546396,1.321261,0.163478
25%,4.410775,13.526323,5.055388,1.953060
50%,4.570435,14.914761,10.484369,3.329395
75%,4.920659,17.537580,15.274448,4.496819
max,5.268321,68.519989,24.583452,6.593685


In [ ]:
mse_tensorflow_df.describe()

,baseline,dense,convolutional,LSTM
count,28.000000,28.000000,28.000000,28.000000
mean,0.033793,0.151559,0.061777,0.006400
std,0.005115,0.321738,0.061794,0.005215
min,0.021251,0.040944,0.000865,0.000012
25%,0.030550,0.068143,0.012387,0.001746
50%,0.034855,0.090444,0.042196,0.006359
75%,0.037118,0.115979,0.090197,0.011165
max,0.041895,1.785194,0.234939,0.016295
